In [ ]:
# Gerekli kütüphaneler
!pip install transformers datasets pydot pandas evaluate bert_score --quiet

import torch
import pandas as pd
import pydot
from transformers import T5Tokenizer, T5ForConditionalGeneration

# -------------------------------
# 1 Hugging Face modelini yükle
# -------------------------------
model_id = "zezomii/akis-t5"  # HF Hub repoadın
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id).to(device)

# ---------------------------------
# 2 Modelden tahmin alacak fonksiyon
# ---------------------------------
def generate_flow_text(input_text, max_length=512):
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(input_ids, max_length=max_length)
    pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return pred_text

# ---------------------------------
# 3 Model çıktısını DataFrame'e çevir
# ---------------------------------
def parse_model_output_to_df(model_output, flow_id="F1"):
    lines = model_output.strip().split("Adm ")
    data = []

    for line in lines:
        if line.strip() == "":
            continue
        # Örnek: "1: Süreç - 'Müşteri...' , next:2"
        try:
            # Adım numarası
            step_no = int(line.split(":")[0].strip())
            # Tür
            step_type = line.split(":")[1].split("-")[0].strip().lower()
            # Açıklama
            desc_part = line.split("-")[1]
            desc = desc_part.split(", next")[0].replace("'", "").strip()
            # Evet / Hayır
            if "yes-next" in line:
                yes_next = line.split("yes-next:")[1].split(",")[0].strip()
            elif "next:" in line:
                yes_next = line.split("next:")[1].strip()
            else:
                yes_next = "-"
            no_next = line.split("no-next:")[1].strip() if "no-next" in line else "-"

            data.append({
                "Akış ID": flow_id,
                "Adım No": step_no,
                "Adım Açıklaması": desc,
                "Adım Türü": step_type,
                "Evet Durumu (Sonraki Adım)": yes_next,
                "Hayır Durumu (Sonraki Adım)": no_next
            })
        except Exception as e:
            print("Parsing Hatası:", e, "-> Line:", line)
            continue

    df = pd.DataFrame(data)
    return df


# ---------------------------------
# 4 Akış diyagramını oluştur
# ---------------------------------
def create_general_flowchart(df, flow_id):
    flow_data = df[df['Akış ID'] == flow_id]
    graph = pydot.Dot(graph_type="digraph", rankdir="TB")

    shape_dict = {
        "süreç": "box",
        "koşul": "diamond",
        "veritabanı kontrol": "cylinder",
        "veritabanı kaydet": "cylinder",
        "mail gönderme": "parallelogram",
        "web servis": "triangle",
        "başlangıç": "ellipse",
        "bitiş": "ellipse"
    }

    for _, row in flow_data.iterrows():
        step_id = str(row['Adım No'])
        label = row['Adım Açıklaması']
        step_type = row['Adım Türü']
        shape = shape_dict.get(step_type, "rectangle")

        node = pydot.Node(step_id, label=label, shape=shape, style="filled", fillcolor="lightgrey")
        graph.add_node(node)

        # Koşul adımlarında dallar
        if step_type == "koşul":
            if row['Evet Durumu (Sonraki Adım)'] != "-":
                graph.add_edge(pydot.Edge(step_id, str(row['Evet Durumu (Sonraki Adım)']), label="Evet"))
            if row['Hayır Durumu (Sonraki Adım)'] != "-":
                graph.add_edge(pydot.Edge(step_id, str(row['Hayır Durumu (Sonraki Adım)']), label="Hayır"))
        else:
            # Diğer adımlar için sonraki adıma bağla
            next_step = row['Evet Durumu (Sonraki Adım)']
            if next_step != "-":
                graph.add_edge(pydot.Edge(step_id, str(next_step)))

    output_file = f'general_flowchart_{flow_id}.png'
    graph.write_png(output_file)
    print(f"Diyagram {output_file} olarak kaydedildi.")
    return output_file

# ---------------------------------
# 5 Kullanıcıdan girdi al, model tahmini, DataFrame ve diyagram
# ---------------------------------
user_input = input("Akış metnini girin: ")
model_output = generate_flow_text(user_input)

print("\n--- Model Çıktısı ---")
print(model_output)

df = parse_model_output_to_df(model_output, flow_id="F1")
print("\n--- DataFrame ---")
print(df)

create_general_flowchart(df, flow_id="F1")
